## LightGBM算法梳理

1. LightGBM
- LightGBM的起源
- Histogram VS pre-sorted
- leaf-wise VS level-wise
- 特征并行和数据并行
- 顺序访问梯度
- 支持类别特征
- 应用场景
- sklearn参数
- CatBoost(了解)

### 1.LGB
GBDT在面对大数据(in terms of both the number of features
and the number of instances)的时候，因为要对每个特征遍历整个数据集计算信息增益(information gain)，所以会非常耗时(time consuming)，
为了解决以上问题，提出了两种方法
- Gradient-based One-Side Sampling (GOSS). 
较大梯度对于信息增益有正影响；保留梯度大的样本，随即删除梯度小的样本。
- Exclusive Feature Bundling (EFB).
通常在实际应用中，虽然有大量的特征，但特征空间是相当稀疏的(sparse)，这为我们设计一种近乎无损的方法来减少有效特征的数量提供了可能。具体地说，在一个稀疏的特征空间(future space)中，许多特征(几乎)是独占的，即，它们很少同时取非零值。为此，我们设计了一种有效的算法，将最优捆绑(optimal bundling)问题简化为图着色（graph coloring）问题(如果两个特征不互斥，则以特征为顶点，每两个特征边相加)，并用一个常数近似比（constant approximation ratio）的贪婪算法求解。

我们称这种新的基于GOSS和EFB的GBDT算法为LightGBM

![](https://i.loli.net/2019/08/15/43CJ2Mb81xO9far.jpg)

相比XGBoost有如下优点：

- 基于Histogram的决策树算法, 更快的训练速度和更高的效率：LightGBM使用基于直方图的算法。例如，它将连续的特征值分桶(buckets)装进离散的箱子(bins)，这是的训练过程中变得更快。

- 带深度限制的Leaf-wise的叶子生长策略,   LightGBM的分裂节点的方式与XGBoost不一样。LGB避免了对整层节点分裂法，而采用了对增益最大的节点进行深入分解的方法。这样节省了大量分裂节点的资源。

- 更低的内存占用：使用离散的箱子(bins)保存并替换连续值导致更少的内存占用。

- 更高的准确率(相比于其他任何提升算法)：它通过leaf-wise分裂方法产生比level-wise分裂方法更复杂的树，这就是实现更高准确率的主要因素。然而，它有时候或导致过拟合，但是我们可以通过设置 max-depth 参数来防止过拟合的发生。

- 大数据处理能力：相比于XGBoost，由于它在训练时间上的缩减，它同样能够具有处理大数据的能力。

- 支持并行学习

- Cache命中率优化

- 基于直方图的稀疏特征优化

### 2.LightGBM起源

#### LightGBM起源于微软亚洲研究院在NIPS发表的系列论文：

- [Qi Meng, Guolin Ke, Taifeng Wang, Wei Chen, Qiwei Ye, Zhi-Ming Ma, Tie-Yan Liu. “A Communication-Efficient Parallel Algorithm for Decision Tree.” Advances in Neural Information Processing Systems 29 (NIPS 2016), pp. 1279-1287](https://papers.nips.cc/paper/6381-a-communication-efficient-parallel-algorithm-for-decision-tree.pdf)


- [Guolin Ke, Qi Meng, Thomas Finley, Taifeng Wang, Wei Chen, Weidong Ma, Qiwei Ye, Tie-Yan Liu. “LightGBM: A Highly Efficient Gradient Boosting Decision Tree.” Advances in Neural Information Processing Systems 30 (NIPS 2017), pp. 3149-3157](https://papers.nips.cc/paper/6907-lightgbm-a-highly-efficient-gradient-boosting-decision-tree.pdf)


- 之后增加了GPU加速支持，[Huan Zhang, Si Si and Cho-Jui Hsieh. "GPU Acceleration for Large-scale Tree Boosting". SysML Conference, 2018.](https://arxiv.org/abs/1706.08359)

并于2016年10月17日在Github开源,

[Key events](https://github.com/Microsoft/LightGBM/blob/master/docs/Key-Events.md)

### 3.Histogram VS pre-sorted

GBDT是通过拟合残差来学习决策树；GBDT中最耗时的就是在学习决策树过程中寻找最好分割点和分割数据。

- 最流行的算法之一就是预排序(pre-sorted)算法，它为每个特性值预先排序并枚举所有可能的拆分点。这种算法可以精确地找到最佳分割点。但是在训练速度和内存消耗上都inefficient.而且可能存在过拟合。
 - 可能会造成cache miss
   - 对梯度的访问，计算gain的时候会访问梯度，不同feature访问梯度的顺序是不一样，随机的
   - 行号和叶子节点的索引表，防止切分的时候对所有数据切分，和上面一样随机访问
![](https://i.loli.net/2019/08/14/hYytRP49soneuWX.jpg)
- 另一种方法就是histogram-based algorithm，基于直方图算法，这种算法在训练的时候将连续的特征值，分为离散的bins，并用它们构建新的特征直方图。且用更小的byte来存储bins。用直方图做差来得到加速效果。
 - 对梯度访问:不需要对feature排序，所有feature都用同样的方式访问，对梯度访问顺序进行排序。
 - 直方图不需要数据id到叶子号的索引，没有cache miss的问题。在数据量很大的时候顺序访问比随机访问效率大很多
![](https://i.loli.net/2019/08/14/FlCu2txBYdD91HP.jpg)
所以这种算法更在速度和内存消耗上都更高效。

 - It costs $O(#data × #feature)$ for histogram building and $O(#bin × #feature)$ for
split point finding.Since $#bin$ is usually much smaller than #data, histogram building will dominate
the computational complexity. If we can reduce $#data$ or #feature, we will be able to substantially
speed up the training of GBDT.

- 传统的ml算法，不能直接输入类别特征，需要对数据离散化，转化成多维的0/1特征，降低效率。lightgbm通过更改决策树的决策规则，直接支持输入类别特征，不用离散化。
![](https://i.loli.net/2019/08/14/5Tj9H8EVJOQDuZm.jpg)

- histogram 算法不能找到很精确的分割点，训练误差没有 pre-sorted 好。但从实验结果来看， histogram 算法在测试集的误差和 pre-sorted 算法差异并不是很大，甚至有时候效果更好。实际上可能决策树对于分割点的精确程度并不太敏感，而且较“粗”的分割点也自带正则化的效果，再加上boosting算法本身就是弱分类器的集成。


### 4.leaf-wise VS level-wise
lgb带有深度限制的leaf-wise来提高模型精度，且比level-wise高效。叶数量一样的时候leaf-wise可以减少更多的loss，得到更好的精度；但可能会过拟合，比如在小数据集训练，产生比较深的决策树，所以加了深度限制。

- 生长策略

  - Level-wise过一次数据可以同时分裂同一层的叶子，容易进行多线程优化，也好控制模型复杂度，不容易过拟合。但实际上Level-wise是一种低效算法，因为它不加区分的对待同一层的叶子，带来了很多没必要的开销，因为实际上很多叶子的分裂增益较低，没必要进行搜索和分裂。
  ![](https://github.com/microsoft/LightGBM/raw/master/docs/_static/images/level-wise.png)

  - Leaf-wise则是一种更为高效的策略：每次从当前所有叶子中，找到分裂增益最大的一个叶子，然后分裂，如此循环。因此同Level-wise相比，在分裂次数相同的情况下，Leaf-wise可以降低更多的误差，得到更好的精度。对于过拟合，可以通过设置另一个参数 max-depth 来控制它分裂产生的树的最大深度。
  ![](https://github.com/microsoft/LightGBM/raw/master/docs/_static/images/leaf-wise.png)
  convert max_depth to num_leaves

标准是:num_leaves = 2^max_depth

### 5.特征并行和数据并行

#### Optimization in Parallel Learning


LightGBM provides the following parallel learning algorithms.

##### 1. Feature Parallel

- Traditional Algorithm

 Feature parallel aims to parallelize the "Find Best Split" in the decision tree. The procedure of traditional feature parallel is:

 1. Partition data vertically (different machines have different feature set).

 2. Workers find local best split point {feature, threshold} on local feature set.

 3. Communicate local best splits with each other and get the best one.

 4. Worker with best split to perform split, then send the split result of data to other workers.

 5. Other workers split data according to received data.

 The shortcomings of traditional feature parallel:

 -  Has computation overhead, since it cannot speed up "split", whose time complexity is $O(#data)$.
   Thus, feature parallel cannot speed up well when $#data$ is large.

 -  Need communication of split result, which costs about $O(#data / 8)$ (one bit for one data).

- Feature Parallel in LightGBM
![](https://i.loli.net/2019/08/14/ZOVbz1xS4Ncnjd2.jpg)
Since feature parallel cannot speed up well when $#data$ is large, we make a little change: instead of partitioning data vertically, every worker holds the full data.
Thus, LightGBM doesn't need to communicate for split result of data since every worker knows how to split data.
And $#data$ won't be larger, so it is reasonable to hold the full data in every machine.

 The procedure of feature parallel in LightGBM:

 1. Workers find local best split point {feature, threshold} on local feature set.

 2. Communicate local best splits with each other and get the best one.

 3. Perform best split.

 However, this feature parallel algorithm still suffers from computation overhead for "split" when $#data$ is large.
So it will be better to use data parallel when $#data$ is large.

##### 2. Data Parallel

- Traditional Algorithm

 Data parallel aims to parallelize the whole decision learning. The procedure of data parallel is:

 1. Partition data horizontally.

 2. Workers use local data to construct local histograms.

 3. Merge global histograms from all local histograms.

 4. Find best split from merged global histograms, then perform splits.

 The shortcomings of traditional data parallel:

 -  High communication cost.
   If using point-to-point communication algorithm, communication cost for one machine is about $O(#machine * #feature * #bin)$.
  
   If using collective communication algorithm (e.g. "All Reduce"), communication cost is about $O(2 * #feature * #bin)$ (check cost of "All Reduce" in chapter 4.5 at `[9] <#references>`__).

- Data Parallel in LightGBM
![](https://i.loli.net/2019/08/14/ai15HPMZ6Cc2ugU.jpg)
 We reduce communication cost of data parallel in LightGBM:

 1. Instead of "Merge global histograms from all local histograms", LightGBM uses "Reduce Scatter" to merge histograms of different (non-overlapping) features for different workers.

   Then workers find the local best split on local merged histograms and sync up the global best split.

 2. As aforementioned, LightGBM uses histogram subtraction to speed up training.
 
   Based on this, we can communicate histograms only for one leaf, and get its neighbor's histograms by subtraction as well.

 All things considered, data parallel in LightGBM has time complexity $O(0.5 * #feature * #bin)$.

##### 3. Voting Parallel
![](https://i.loli.net/2019/08/14/IVwn7QtDeghLZ4J.jpg)
 Voting parallel further reduces the communication cost in `Data Parallel <#data-parallel>`__ to constant cost.
It uses two-stage voting to reduce the communication cost of feature histograms\ `[10] <#references>`__.

[中文翻译](https://blog.csdn.net/ictcxq/article/details/78736442)

### 6.顺序访问梯度

xgb等
- 对梯度的访问：在计算gain的时候需要利用梯度，对于不同的特征，访问梯度的顺序是不一样的，并且是随机的
- 对于索引表的访问：预排序算法使用了行号和叶子节点号的索引表，防止数据切分的时候对所有的特征进行切分。同访问梯度一样，所有的特征都要通过访问这个索引表来索引。
这两个操作都是随机的访问，会给系统性能带来非常大的下降。

lgb
- 对梯度访问不需要对feature排序，所有feature都用同样的方式访问，对梯度访问顺序进行排序。

- 直方图不需要数据id到叶子号的索引，没有[cachemiss](https://www.cnblogs.com/jokerjason/p/10711022.html)的问题。在数据量很大的时候顺序访问比随机访问效率大很多

### 7.支持类别特征

传统的ml算法，不能直接输入类别特征，需要对数据离散化，转化成多维的0/1特征，降低效率。lightgbm通过更改决策树的决策规则，直接支持输入类别特征，不用离散化。
![](https://i.loli.net/2019/08/14/5Tj9H8EVJOQDuZm.jpg)

[LightGBM源码阅读+理论分析（处理特征类别，缺省值的实现细节）](
https://blog.csdn.net/weixin_42001089/article/details/85343332)


### 8.应用场景
和xgb一样，适合大数据量(高维度、高数据量)处理的场景

### 9.sklearn参数
https://juejin.im/post/5b76437ae51d45666b5d9b05
https://lightgbm.apachecn.org/#/docs/6

[参数调优](https://zhuanlan.zhihu.com/p/35645973)

[官方给出的竞赛例子](https://github.com/microsoft/LightGBM/tree/master/examples)

### 10.CatBoost(了解)
http://learningsys.org/nips17/assets/papers/paper_11.pdf

convert max_depth to num_leaves

num_leaves = 2^max_depth